# Working with the Chinook Database

We are going to make some analysis of the Chinook Database to answer interesting/useful question about its data. In summary, this is a Database with data from an online music store. We will briefly explore its data before going into the questions we want to answer. More information about the Database can be found on the Common Databases folder in this repository.

Our first step is to connect with the database:

In [1]:
import os
import sqlite3

COMMON_DATABASES_FOLDER = os.path.join(os.path.join(os.path.abspath(".."), "Common_Databases"), "chinook")
CHINOOK_PATH = os.path.join(COMMON_DATABASES_FOLDER, "chinook.db")

conn = sqlite3.connect(CHINOOK_PATH)


When working with SQLite, we can list all the tables in a Database using a SELECT statement.

The Chinook Database is composed by 11 tables. In the cell below we see them all, as well as some tables started with "sqlite_", which are automatically generated by SQLite. We only need the original Chinook tables, and so we will filter the results. One should notice that the list returned when calling the fetchall() function is a list of tuple objects, and not a list of strings.

In [2]:
cursor = conn.execute('''SELECT name FROM sqlite_master WHERE type='table';''')

cursor.fetchall()

[('albums',),
 ('sqlite_sequence',),
 ('artists',),
 ('customers',),
 ('employees',),
 ('genres',),
 ('invoices',),
 ('invoice_items',),
 ('media_types',),
 ('playlists',),
 ('playlist_track',),
 ('tracks',),
 ('sqlite_stat1',)]

Retrieving only the original Chinook tables names can be done in the following way:

In [3]:
tablesNames = [
    table[0] for table in conn.execute('''SELECT name FROM sqlite_master WHERE type='table';''').fetchall() 
    if not "sqlite_" in table[0]
]
tablesNames

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoices',
 'invoice_items',
 'media_types',
 'playlists',
 'playlist_track',
 'tracks']